In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cma
from es import SimpleGA, CMAES, PEPG, OpenES

def sigmoid(x):
    z = 1/(1 + np.exp(-x)) 
    return z

def BCE_loss(y,p):
    return np.sum(-y*np.log(p) - (1 - y)*np.log(1 - p))


def binarize(x):
    res = x > 0.5
    return res.astype(int)


def sigmoid(x):
  return 1 / (1 + np.exp(-x))

def relu(x):
  return np.maximum(x, 0)

def passthru(x):
  return x

# useful for discrete actions
def softmax(x):
  e_x = np.exp(x - np.max(x))
  return e_x / e_x.sum(axis=0)

# useful for discrete actions
def sample(p):
  return np.argmax(np.random.multinomial(1, p))


class RNNCell:
  def __init__(self, input_size, weight, bias):
    self.input_size=input_size
    self.weight = weight
    self.bias = bias
  def __call__(self, x, h):
    concat = np.concatenate((x, h), axis=1)
    hidden = np.matmul(concat, self.weight)+self.bias
    return np.tanh(hidden)

class RNNModel:
  def __init__(self):
    

    self.hidden_size = 10

    self.layer_1 = 10
    self.layer_2 = 10

    self.rnn_mode = True

    self.input_size = 1
    self.output_size = 1


    self.shapes = [ (self.input_size + self.hidden_size, 1*self.hidden_size), # RNN weights
                    (self.input_size + self.hidden_size, self.layer_1),# predict actions output
                    (self.layer_1, self.output_size)] # predict actions output

    self.weight = []
    self.bias = []
    self.param_count = 0

    idx = 0
    for shape in self.shapes:
      self.weight.append(np.zeros(shape=shape))
      self.bias.append(np.zeros(shape=shape[1]))
      self.param_count += (np.product(shape) + shape[1])
      idx += 1
    
    self.init_h = np.zeros((1, self.hidden_size))
    self.h = self.init_h
    self.param_count += 1*self.hidden_size
    
    self.rnn = RNNCell(self.input_size, self.weight[0], self.bias[0])

  def reset(self):
    self.h = self.init_h


  def get_action(self, x):
    obs = x.reshape(1, self.input_size)

    # update rnn:
    #update_obs = np.concatenate([obs, action], axis=1)
    self.h = self.rnn(x, binarize(sigmoid(self.h)))

    # get action
    x = np.concatenate([x, self.h], axis=1)

    # calculate action using 2 layer network from output
    hidden = np.tanh(np.matmul(x, self.weight[1]) + self.bias[1])
    action = sigmoid(np.matmul(hidden, self.weight[2]) + self.bias[2])

    return action[0]

  def set_model_params(self, model_params):
    pointer = 0
    for i in range(len(self.shapes)):
      w_shape = self.shapes[i]
      b_shape = self.shapes[i][1]
      s_w = np.product(w_shape)
      s = s_w + b_shape
      chunk = np.array(model_params[pointer:pointer+s])
      self.weight[i] = chunk[:s_w].reshape(w_shape)
      self.bias[i] = chunk[s_w:].reshape(b_shape)
      pointer += s
    # rnn states
    s = self.hidden_size
    self.init_h = model_params[pointer:pointer+s].reshape((1, self.hidden_size))
    self.h = self.init_h
    self.rnn = RNNCell(self.input_size, self.weight[0], self.bias[0])

  def load_model(self, filename):
    with open(filename) as f:    
      data = json.load(f)
    print('loading file %s' % (filename))
    self.data = data
    model_params = np.array(data[0]) # assuming other stuff is in data
    self.set_model_params(model_params)

  def get_random_model_params(self, stdev=0.1):
    return np.random.randn(self.param_count)*stdev

    


In [2]:
model = RNNModel()
model.get_action(np.array([[4]]))


array([0.5])

In [3]:
model = RNNModel()
NPARAMS = model.param_count    # make this a 100-dimensinal problem.
NPOPULATION = 401    # use population size of 101.
MAX_ITERATION = 4000 # run each solver for 5000 generations.


In [4]:
def recurrency_label(seq_len):
    
    labels = []
    X = []

    X = np.zeros([seq_len,1])
    #X[0,:] = 1.0
    for ii in range(seq_len):
        if ii % 30 == 0:
            labels.append(np.ones([1,1]))

        else:
            labels.append(np.zeros([1,1]))

    return X, np.concatenate(labels, axis=0)

def evluate_func(model, params):
    model.set_model_params(params)
    model.reset()
    loss_cum = 0
    Xs, labels  = recurrency_label(32)

    for x, label in zip(Xs,labels):
        

        x = np.array([x])
        pred = model.get_action(x)
        
        #print(label, pred)
        loss = BCE_loss(label, pred)
        loss_cum += loss
    #print(loss_cum)
    return loss_cum

In [6]:


# defines genetic algorithm solver
ga = SimpleGA(NPARAMS,                # number of model parameters
               sigma_init=0.5,        # initial standard deviation
               popsize=NPOPULATION,   # population size
               elite_ratio=0.05,       # percentage of the elites
               forget_best=False,     # forget the historical best elites
               weight_decay=0.00,     # weight decay coefficient
              )

In [7]:
fit_func = evluate_func
# defines a function to use solver to solve fit_func
def test_solver(solver):
    history = []
    for j in range(MAX_ITERATION):
        solutions = solver.ask()
        fitness_list = np.zeros(solver.popsize)
        #print(solutions)
        for i in range(solver.popsize):
            fitness_list[i] = -fit_func(model,solutions[i])
            #print(fit_func(model,solutions[i]))
            

        solver.tell(fitness_list)
        result = solver.result() # first element is the best solution, second element is the best fitness
        history.append(result[1])
        if (j+1) % 10 == 0:
            print("fitness at iteration", (j+1), result[1])
    print("local optimum discovered by solver:\n", result[0])
    print("fitness score at this local optimum:", result[1])
    return history

ga_history = test_solver(ga)

KeyboardInterrupt: 

In [ ]:
cmaes = CMAES(NPARAMS,
              popsize=NPOPULATION,
              weight_decay=0.0,
              sigma_init = 2.0
          )
cma_history = test_solver(cmaes)

(200_w,401)-aCMA-ES (mu_w=103.6,w_1=2%) in dimension 261 (seed=363778, Wed Nov 25 19:10:05 2020)
fitness at iteration 10 -2.8570919682928655
fitness at iteration 20 -2.8570919682928655
fitness at iteration 30 -2.790697508833722
fitness at iteration 40 -2.790697508833722
fitness at iteration 50 -2.790697508833722
fitness at iteration 60 -2.790697508833722
fitness at iteration 70 -2.790697508833722
fitness at iteration 80 -2.790697508833722
fitness at iteration 90 -2.790697508833722
fitness at iteration 100 -2.790697508833722
fitness at iteration 110 -2.790697508833722
fitness at iteration 120 -2.790697508833722


/shared/gabriele/miniconda/envs/ARC/lib/python3.6/site-packages/ipykernel_launcher.py:11: RuntimeWarning: divide by zero encountered in log
  # This is added back by InteractiveShellApp.init_path()
/shared/gabriele/miniconda/envs/ARC/lib/python3.6/site-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in multiply
  # This is added back by InteractiveShellApp.init_path()
/shared/gabriele/miniconda/envs/ARC/lib/python3.6/site-packages/cma/utilities/utils.py:347: UserWarning: function values with index [128]/[] are nan/None and will be set to the median value 11.19441174618478 (class=CMAEvolutionStrategy method=ask iteration=128)
  ')')


fitness at iteration 130 -2.790697508833722


/shared/gabriele/miniconda/envs/ARC/lib/python3.6/site-packages/cma/utilities/utils.py:347: UserWarning: function values with index [297, 303]/[] are nan/None and will be set to the median value 12.212784761958725 (class=CMAEvolutionStrategy method=ask iteration=130)
  ')')
/shared/gabriele/miniconda/envs/ARC/lib/python3.6/site-packages/cma/utilities/utils.py:347: UserWarning: function values with index [65]/[] are nan/None and will be set to the median value 11.004376997301527 (class=CMAEvolutionStrategy method=ask iteration=134)
  ')')
/shared/gabriele/miniconda/envs/ARC/lib/python3.6/site-packages/cma/utilities/utils.py:347: UserWarning: function values with index [172]/[] are nan/None and will be set to the median value 11.60356624054215 (class=CMAEvolutionStrategy method=ask iteration=138)
  ')')
/shared/gabriele/miniconda/envs/ARC/lib/python3.6/site-packages/cma/utilities/utils.py:347: UserWarning: function values with index [288]/[] are nan/None and will be set to the median val

fitness at iteration 140 -2.790697508833722


/shared/gabriele/miniconda/envs/ARC/lib/python3.6/site-packages/cma/utilities/utils.py:347: UserWarning: function values with index [138, 180, 220, 319]/[] are nan/None and will be set to the median value 11.974253470998862 (class=CMAEvolutionStrategy method=ask iteration=140)
  ')')
/shared/gabriele/miniconda/envs/ARC/lib/python3.6/site-packages/cma/utilities/utils.py:347: UserWarning: function values with index [12, 124, 205, 367]/[] are nan/None and will be set to the median value 13.181492911751938 (class=CMAEvolutionStrategy method=ask iteration=141)
  ')')
/shared/gabriele/miniconda/envs/ARC/lib/python3.6/site-packages/cma/utilities/utils.py:347: UserWarning: function values with index [53, 358]/[] are nan/None and will be set to the median value 14.434754484830634 (class=CMAEvolutionStrategy method=ask iteration=142)
  ')')
/shared/gabriele/miniconda/envs/ARC/lib/python3.6/site-packages/cma/utilities/utils.py:347: UserWarning: function values with index [137, 167, 196, 208, 322]

In [10]:
# defines PEPG (NES) solver
pepg = PEPG(NPARAMS,                         # number of model parameters
            sigma_init=0.5,                  # initial standard deviation
            learning_rate=0.1,               # learning rate for standard deviation
            learning_rate_decay=0.0,       # don't anneal the learning rate
            popsize=NPOPULATION,             # population size
            average_baseline=False,          # set baseline to average of batch
            weight_decay=0.00,            # weight decay coefficient
            rank_fitness=False,           # use rank rather than fitness numbers
            forget_best=False)   
pepg_history = test_solver(pepg)

fitness at iteration 10 -5.264003048430048
fitness at iteration 20 -5.264003048430048
fitness at iteration 30 -5.264003048430048
fitness at iteration 40 -5.151312741968082
fitness at iteration 50 -5.151312741968082


KeyboardInterrupt: 

In [ ]:
solutions = ga.ask()
evluate_func(model, solutions[0])

In [165]:
X = model.forward(np.ones([1,12]))
#model.forward(X[0])
X.shape

(1, 11)

In [49]:
model.lyr_shapes

[array([20, 12]), array([20, 20]), array([11, 20])]

In [50]:
model.num_params()

860

In [125]:

def shellSort(arr, k): 
  
    # Start with a big gap, then reduce the gap 
    n = len(arr) 

    
    gap = (k -1)*(k-1)
  
    # Do a gapped insertion sort for this gap size. 
    # The first gap elements a[0..gap-1] are already in gapped  
    # order keep adding one more element until the entire array 
    # is gap sorted 
    while gap > 0:
        print('GAP:', gap)
        for i in range(gap,n): 
  
            # add a[i] to the elements that have been gap sorted 
            # save a[i] in temp and make a hole at position i 
            temp = arr[i] 
  
            # shift earlier gap-sorted elements up until the correct 
            # location for a[i] is found 
            j = i 
            while  j >= gap and arr[j-gap] >temp: 
                arr[j] = arr[j-gap] 
                j -= gap 
  
            # put temp (the original a[i]) in its correct location 
            arr[j] = temp
        print(arr)
        k = k - 1
        gap = (k -1)*(k-1)

  
  
# Driver code to test above 
arr = [93, 27, 71, 94, 68, 12, 35, 22, 25, 43]
  
n = len(arr) 
  
shellSort(arr, len(arr)) 
  

# This code is contributed by Mohit Kumra 

GAP: 81
[93, 27, 71, 94, 68, 12, 35, 22, 25, 43]
GAP: 64
[93, 27, 71, 94, 68, 12, 35, 22, 25, 43]
GAP: 49
[93, 27, 71, 94, 68, 12, 35, 22, 25, 43]
GAP: 36
[93, 27, 71, 94, 68, 12, 35, 22, 25, 43]
GAP: 25
[93, 27, 71, 94, 68, 12, 35, 22, 25, 43]
GAP: 16
[93, 27, 71, 94, 68, 12, 35, 22, 25, 43]
GAP: 9
[43, 27, 71, 94, 68, 12, 35, 22, 25, 93]
GAP: 4
[25, 12, 35, 22, 43, 27, 71, 94, 68, 93]
GAP: 1
[12, 22, 25, 27, 35, 43, 68, 71, 93, 94]
